In [3]:
from ucimlrepo import fetch_ucirepo 

In [4]:
# fetch dataset 
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697) 
  
# data (as pandas dataframes) 
X = predict_students_dropout_and_academic_success.data.features 
y = predict_students_dropout_and_academic_success.data.targets 


In [5]:
X.shape

(4424, 36)

In [6]:
X.columns

Index(['Marital Status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd s

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 36 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital Status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

In [8]:
y_copy = y.copy()

# Update the 'Target' column in the copy
y_copy['Target'] = y_copy['Target'].map({
    'Dropout': 0,
    'Enrolled': 1,
    'Graduate': 2
})

# Now 'y_copy' contains the modified 'Target' column
# If you want to update the original 'y' DataFrame, you can do:
y.loc[:, 'Target'] = y_copy['Target']

In [9]:
import numpy as np

In [10]:
student_x = X
student_y = y

In [11]:
from sklearn.model_selection import train_test_split
student_x_train, student_x_test, student_y_train, student_y_test = train_test_split(student_x, student_y, test_size=1/3, random_state=42)
student_X_train, student_X_val, student_Y_train, student_y_val = train_test_split(student_x_train, student_y_train, test_size=1/3, random_state=42)

In [12]:
import pandas as pd 
student_y_train = student_y_train.astype('int32')
student_Y_train = student_Y_train.astype('int32')
student_y_val = student_y_val.astype('int32')
student_y_test = student_y_test.astype('int32')

student_y_train = pd.to_numeric(student_y_train['Target']).values
student_Y_train = pd.to_numeric(student_Y_train['Target']).values
student_y_test = pd.to_numeric(student_y_test['Target']).values
student_y_val = pd.to_numeric(student_y_val['Target']).values

In [13]:
import util
from importlib import reload
reload(util)
from util import Utils

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [20]:
student_x_train

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
1095,1,1,1,9853,1,1,125.0,1,37,37,...,0,0,6,7,3,10.666667,0,15.5,2.8,-4.06
3507,1,1,1,9670,1,1,131.0,1,1,1,...,0,0,5,5,5,11.600000,0,9.4,-0.8,-3.12
4404,2,39,1,9130,1,19,133.1,1,2,1,...,0,0,6,9,2,10.500000,0,16.2,0.3,-0.92
1320,1,1,5,9238,1,3,135.0,1,1,38,...,0,0,6,6,6,12.333333,0,10.8,1.4,1.74
2716,2,39,2,9556,1,1,110.0,1,19,38,...,0,0,8,8,0,0.000000,6,10.8,1.4,1.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,1,1,2,9070,1,1,134.0,1,1,19,...,1,0,6,11,4,11.333333,0,13.9,-0.3,0.79
466,1,1,1,9853,1,1,133.0,1,1,38,...,0,4,8,8,8,12.625000,0,15.5,2.8,-4.06
3092,1,1,1,8014,0,1,100.0,1,38,19,...,0,0,6,6,6,12.500000,0,12.7,3.7,-1.70
3772,1,1,4,9238,1,1,127.0,1,1,38,...,0,0,6,7,6,13.428571,0,11.1,0.6,2.02


In [25]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

best_student_model = util.Utils.find_best_rf_parameters(student_x_train, student_y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/home/ahmad/AI_Lab/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/ahmad/AI_Lab/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/ahmad/AI_Lab/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/ahmad/AI_Lab/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/ahmad/AI_Lab/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/ahmad/AI_Lab/lib/python3.10/s


All Results:
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'gini'} with accuracy: 0.7741607324516785
{'n_estimators': 100, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy'} with accuracy: 0.7741607324516785
{'n_estimators': 10, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'entropy'} with accuracy: 0.7524584604950831
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 50, 'criterion': 'gini'} with accuracy: 0.7707697524584605
{'n_estimators': 10, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'gini'} with accuracy: 0.7487283825025433
{'n_estimators': 100, 'max_features': 'log2', 'max_depth': 50, 'criterion': 'gini'} with accuracy: 0.7833163784333673
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy'} with accuracy: 0.7714479484571042
{'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 100, 'criterion': 'entropy'} with accuracy: 0.7782299084435401
{'n_estimators': 100, 'max_featu

In [26]:
Utils.accuracy_measure_rf(student_x_train, student_y_train.ravel(), student_x_test, student_y_test.ravel(), n_estimators = 100, criterion = 'entropy', max_depth = 50, max_features = 'log2')

--- 0.38927173614501953 seconds ---


(0.7674576271186441,
 (array([0.81896552, 0.5859375 , 0.76670442]),
  array([0.77709611, 0.27675277, 0.94685315]),
  array([0.79748164, 0.37593985, 0.84730914]),
  array([489, 271, 715])))

In [27]:
best_xgb_model = Utils.find_best_xgb_parameters(student_X_train, student_Y_train, student_X_val, student_y_val)

[CV] END criterion=gini, max_depth=100, max_features=log2, n_estimators=100; total time=   0.1s
[CV] END criterion=entropy, max_depth=10, max_features=log2, n_estimators=50; total time=   0.3s
[CV] END criterion=gini, max_depth=100, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END criterion=gini, max_depth=50, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=gini, max_depth=10, max_features=sqrt, n_estimators=100; total time=   0.0s
[CV] END criterion=entropy, max_depth=50, max_features=log2, n_estimators=10; total time=   0.0s
[CV] END criterion=gini, max_depth=100, max_features=log2, n_estimators=10; total time=   0.0s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, n_estimators=50; total time=   0.3s
[CV] END criterion=entropy, max_depth=100, max_features=log2, n_estimators=100; total time=   0.4s
[CV] END criterion=entropy, max_depth=10, max_features=sqrt, n_estimators=50; total time=   0.4s
[CV] END criterion=entropy, max_dept

In [ ]:
Utils.accuracy_measure_xgb(student_x_train, student_y_train.ravel(), student_x_test, student_y_test.ravel(), n_estimators = 100, learning_rate = 0.1, max_depth = 5)